In [1]:
import os
import pandas as pd

from ftplib import FTP

In [20]:
data_folder = f"../data"

sample_filename = "muestreos_parcelas.parquet"

df_samples = pd.read_parquet(f"{data_folder}/{sample_filename}")

df_samples.info()

<class 'pandas.core.frame.DataFrame'>
Index: 581793 entries, 0 to 581792
Data columns (total 61 columns):
 #   Column                                              Non-Null Count   Dtype         
---  ------                                              --------------   -----         
 0   generated_muestreos                                 581793 non-null  datetime64[us]
 1   codparcela                                          581793 non-null  category      
 2   provincia                                           581793 non-null  string        
 3   municipio                                           581793 non-null  string        
 4   fecha                                               581793 non-null  datetime64[us]
 5   campaña                                             581793 non-null  uint16        
 6   poligono                                            581731 non-null  UInt16        
 7   parcela                                             581763 non-null  UInt16        
 8  

In [53]:
df_samples.head(20).to_csv("sample_parcelas.txt", sep="\t")

municipio
ubeda           25677
loja            19320
antequera       18170
iznalloz        16997
puente genil    16081
                ...  
espeluy            13
olivares           12
alamedilla          8
oria                6
turrillas           2
Name: count, Length: 326, dtype: Int64

In [5]:
geographic_colums = [
       'generated_muestreos',
       'fecha',
       'codparcela',
       '102_coordenada_x_(utm)',
       '103_coordenada_y_(utm)',
       '104_altitud_(m)',
       '201_superf_cultivada_en_la_parcela_agrícola_(ha)',
       '203_orientación'
       ]

df_space_samples = df_samples[geographic_colums].copy()

df_space_samples.info()


<class 'pandas.core.frame.DataFrame'>
Index: 581793 entries, 0 to 581792
Data columns (total 8 columns):
 #   Column                                            Non-Null Count   Dtype         
---  ------                                            --------------   -----         
 0   generated_muestreos                               581793 non-null  datetime64[us]
 1   fecha                                             581793 non-null  datetime64[us]
 2   codparcela                                        581793 non-null  category      
 3   102_coordenada_x_(utm)                            570857 non-null  float64       
 4   103_coordenada_y_(utm)                            570759 non-null  float64       
 5   104_altitud_(m)                                   389024 non-null  float32       
 6   201_superf_cultivada_en_la_parcela_agrícola_(ha)  530584 non-null  float32       
 7   203_orientación                                   244094 non-null  category      
dtypes: category(2), date

In [19]:
df_space_samples.groupby(df_space_samples.codparcela)[["fecha", "codparcela"]].apply("min", "max")

/tmp/ipykernel_18295/4058399820.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_space_samples.groupby(df_space_samples.codparcela)[["fecha", "codparcela"]].apply("min", "max")


""
codparcela
000-00000-00
000-00000-00-0
000-00900-00-U0
000-00999-00-00
000-00999-00-FC
...
R-879
RAIF-2
RAIF-3


In [29]:
group_dates =df_space_samples.groupby("codparcela")["fecha"].agg([('min_fecha', 'min'), ('max_fecha', 'max')]).reset_index()
group_dates

/tmp/ipykernel_18295/3673216942.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_dates =df_space_samples.groupby("codparcela")["fecha"].agg([('min_fecha', 'min'), ('max_fecha', 'max')]).reset_index()


,codparcela,min_fecha,max_fecha
0,000-00000-00,2015-08-25,2015-09-30
1,000-00000-00-0,2015-08-24,2015-09-29
2,000-00900-00-U0,2006-04-24,2009-11-16
3,000-00999-00-00,2006-05-02,2009-11-18
4,000-00999-00-FC,2006-05-02,2006-10-31
...,...,...,...
5234,R-879,2007-03-12,2007-10-31
5235,RAIF-2,2020-02-27,2020-03-19
5236,RAIF-3,2020-02-27,2020-03-19
5237,RAIF-4,2020-02-27,2020-03-19


In [35]:
group_coords =df_space_samples.groupby("codparcela")[["102_coordenada_x_(utm)", "103_coordenada_y_(utm)"]].apply("first").reset_index()


/tmp/ipykernel_18295/1485127070.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_coords =df_space_samples.groupby("codparcela")[["102_coordenada_x_(utm)", "103_coordenada_y_(utm)"]].apply("first").reset_index()


,codparcela,102_coordenada_x_(utm),103_coordenada_y_(utm)
0,000-00000-00,276171.00,4082574.00
1,000-00000-00-0,232382.86,4072834.94
2,000-00900-00-U0,449521.00,4124050.00
3,000-00999-00-00,449772.00,4123208.00
4,000-00999-00-FC,449772.00,4123208.00
...,...,...,...
5234,R-879,NaN,NaN
5235,RAIF-2,NaN,NaN
5236,RAIF-3,NaN,NaN
5237,RAIF-4,NaN,NaN


In [47]:
result = pd.merge(group_dates, group_coords, on='codparcela', how='inner')

result["days"] = (result["max_fecha"] - result["min_fecha"]).dt.days

result.to_csv("parcelas.txt", "\t")

In [55]:
result.sort_values(by="103_coordenada_y_(utm)", ascending=True).dropna()

,codparcela,min_fecha,max_fecha,102_coordenada_x_(utm),103_coordenada_y_(utm),days
2539,014-00067-01-19,2021-03-10,2021-03-31,3.022360e+02,4.092371e+00,21
2234,012-00013-02-38,2021-03-10,2021-03-31,3.022530e+02,4.092610e+00,21
2840,016-00168-01-19,2021-03-10,2021-03-31,3.007210e+02,4.093063e+00,21
2898,017-00019-02-38,2021-03-10,2021-03-31,2.989240e+02,4.095014e+00,21
1676,009-00001-01-38,2021-03-10,2021-03-31,3.055020e+02,4.095648e+00,21
...,...,...,...,...,...,...
2870,016-00487-05,2018-04-18,2020-11-17,4.155309e+11,4.092620e+12,944
2947,017-00145-01,2018-04-18,2020-11-17,4.111982e+11,4.095019e+12,944
2150,011-00188-01,2018-04-18,2020-11-17,4.104585e+11,4.096575e+12,944
2856,016-00209-02,2015-08-26,2015-11-11,4.230182e+11,4.119741e+12,77


In [52]:
result[result["102_coordenada_x_(utm)"].isna()]

,codparcela,min_fecha,max_fecha,102_coordenada_x_(utm),103_coordenada_y_(utm),days
8,001-00002-01,2019-07-01,2019-10-14,NaN,NaN,105
40,001-00018-00-10,2009-10-26,2009-10-26,NaN,NaN,0
51,001-00026-00-00,2008-10-29,2009-10-26,NaN,NaN,362
74,001-00043-03-00,2016-08-30,2016-10-03,NaN,NaN,34
115,001-00109-01-00,2009-03-18,2011-11-14,NaN,NaN,971
...,...,...,...,...,...,...
5234,R-879,2007-03-12,2007-10-31,NaN,NaN,233
5235,RAIF-2,2020-02-27,2020-03-19,NaN,NaN,21
5236,RAIF-3,2020-02-27,2020-03-19,NaN,NaN,21
5237,RAIF-4,2020-02-27,2020-03-19,NaN,NaN,21


In [32]:
municipios_list = df_samples["municipio"].unique()

len(municipios_list)

326

In [38]:
"espartinas" in municipios_list

True

In [5]:
df_utm = pd.read_csv(f"../data/ngbe.csv", sep=";")
df_utm.head()



/tmp/ipykernel_44297/3120916277.py:1: DtypeWarning: Columns (1,3,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df_utm = pd.read_csv(f"../data/ngbe.csv", sep=";")


,id,codigo_ngbe,codigo_ngbe_text,nombre_extendido,identificador_geografico,nombre_alternativo_2,nombre_alternativo_3,nombre_variante_1,nombre_variante_2,nombre_variante_3,...,estatus_variante_3,long_etrs89_regcan95,lat_etrs89_regcan95,huso_etrs89_regcan95,x_utm_etrs89_regcan95,y_utm_etrs89_regcan95,hojamtn_25,codigo_ine,provincias_nombre,provincias_id
0,3000000,1.0,División administrativa.Estado,NaN,España,NaN,NaN,NaN,NaN,NaN,...,NaN,-3.685424,40.307923,30.0,441755.83,4462159.91,0582c2,00000,Estatal,00
1,2720806,1.1.1,División administrativa.Comunidad autónoma,Cataluña/Catalunya,Cataluña,Catalunya,NaN,NaN,NaN,NaN,...,NaN,2.166687,41.383484,31.0,430324.55,4581663.81,0421c3,"08000,17000,25000,43000","Tarragona,Barcelona,Lleida,Girona","43,08,25,17"
2,2720814,1.1.1,División administrativa.Comunidad autónoma,País Vasco/Euskadi,País Vasco,Euskadi,NaN,NaN,NaN,NaN,...,NaN,-2.666687,42.850098,30.0,527233.75,4744222.52,0112c4,"01000, 20000, 48000","Bizkaia,Araba/Álava,Gipuzkoa","48,01,20"
3,2720800,1.1.1,División administrativa.Comunidad autónoma,NaN,Andalucía,NaN,NaN,NaN,NaN,NaN,...,NaN,-5.983276,37.383484,30.0,235867.22,4141591.98,0984c4,"04000,11000,14000,18000,21000,23000,29000,41000","Málaga,Granada,Almería,Cádiz,Jaén,Córdoba,Huel...","29,18,04,11,23,14,21,41"
4,2720804,1.1.1,División administrativa.Comunidad autónoma,NaN,Castilla y León,NaN,NaN,NaN,NaN,NaN,...,NaN,-4.716674,41.650086,30.0,357050.36,4612350.23,0372c1,"05000,09000,24000,34000,37000,40000,42000,4700...","Segovia,Ávila,Palencia,Valladolid,Burgos,León,...","40,05,34,47,09,24,49,42,37"


In [41]:
municipio_list = [
       'Poblaciones y construcciones.Capital de municipio',
       'Poblaciones y construcciones.Capital de EATIM',
       'Poblaciones y construcciones.Entidad colectiva']


df_utm["identificador_geografico"] = df_utm["identificador_geografico"].str.lower()

df_municipios = df_utm[df_utm["identificador_geografico"].isin(municipios_list)]

In [45]:
df_municipios.drop_duplicates(subset="identificador_geografico")

,id,codigo_ngbe,codigo_ngbe_text,nombre_extendido,identificador_geografico,nombre_alternativo_2,nombre_alternativo_3,nombre_variante_1,nombre_variante_2,nombre_variante_3,...,estatus_variante_3,long_etrs89_regcan95,lat_etrs89_regcan95,huso_etrs89_regcan95,x_utm_etrs89_regcan95,y_utm_etrs89_regcan95,hojamtn_25,codigo_ine,provincias_nombre,provincias_id
26,2721827,1.2,División administrativa.Provincia,NaN,granada,NaN,NaN,NaN,NaN,NaN,...,NaN,-3.600019,37.176419,30.0,446736.015389,4.114612e+06,1009c4,18087,Granada,18
108,2709286,1.3,División administrativa.Municipio,NaN,alameda,NaN,NaN,NaN,NaN,NaN,...,NaN,-4.660269,37.209459,30.0,352676.380947,4.119400e+06,1006c4,29001000000,Málaga,29
146,2709289,1.3,División administrativa.Municipio,NaN,alfarnatejo,NaN,NaN,NaN,NaN,NaN,...,NaN,-4.272012,36.979574,30.0,386788.543533,4.093362e+06,1039c2,29004000000,Málaga,29
232,2708424,1.3,División administrativa.Municipio,NaN,villanueva del arzobispo,NaN,NaN,NaN,NaN,NaN,...,NaN,-3.011866,38.169629,30.0,498960.592428,4.224636e+06,0886c4,23097000000,Jaén,23
522,2706971,1.3,División administrativa.Municipio,NaN,baena,NaN,NaN,NaN,NaN,NaN,...,NaN,-4.328959,37.613188,30.0,382708.067290,4.163729e+06,0967c2,14007000000,Córdoba,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114333,2821407,2.1.9,Poblaciones y construcciones.Entidad singular,NaN,setenil,NaN,NaN,NaN,NaN,NaN,...,NaN,-5.180909,36.862916,30.0,305590.180000,4.081886e+06,1037c3,11034000100,Cádiz,11
678327,496858,4.2.2,Orografía continental y áreas geográficas.Paraje,NaN,jodar,NaN,NaN,NaN,NaN,NaN,...,NaN,-1.153383,38.767883,30.0,660429.030000,4.292638e+06,0819c1,02009,Albacete,02
705104,573628,4.2.2,Orografía continental y áreas geográficas.Paraje,NaN,senes,NaN,NaN,NaN,NaN,NaN,...,NaN,-2.300110,37.165894,30.0,562138.300000,4.113505e+06,1013c4,04082,Almería,04
845692,306942,4.2.2,Orografía continental y áreas geográficas.Paraje,NaN,alora,NaN,NaN,NaN,NaN,NaN,...,NaN,-6.866214,41.014524,29.0,679421.060000,4.542562e+06,0448Bc4,37132,Salamanca,37


In [51]:
len(df_municipios["identificador_geografico"].unique())

218

In [43]:
df_samples.head()

,generated_muestreos,codparcela,provincia,municipio,fecha,campaña,poligono,parcela,recinto,subrecinto,...,402_sensor_climático_asociado,207_riego:_sistema_usual_de_riego,108_u_h_c_a_la_que_pertenece,316_fecha_de_plantación_variedad_secundaria,315_patrón_variedad_secundaria,317_%_superficie_ocupada_variedad_secundaria,306_altura_de_copa_(m),310_patrón_variedad_principal,411_representa_a_la_u_h_c_(si/no),109_sistema_para_el_cumplimiento_gestión_integrada
0,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-04-16,2006,23,109,0,0,...,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-04-27,2006,23,109,0,0,...,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-04,2006,23,109,0,0,...,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-11,2006,23,109,0,0,...,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-17,2006,23,109,0,0,...,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
municipios_list

<StringArray>
[                   'abla',                'abrucena',
                   'antas',     'armuna de almanzora',
                  'finana',          'gallardos  los',
                  'gergal',           'huercal overa',
 'lucainena de las torres',              'nacimiento',
 ...
                'penaflor',                   'pruna',
   'puebla de cazalla  la',           'rinconada  la',
   'roda de andalucia  la',               'rubio  el',
       'sanlucar la mayor',             'saucejo  el',
                  'utrera',  'villanueva de san juan']
Length: 326, dtype: string

In [57]:
ine = df_municipios["identificador_geografico"].unique().tolist()

In [59]:
# Encontrar elementos que no están en ambas listas
elementos_no_comunes = list(set(municipios_list).symmetric_difference(set(ine)))

print(elementos_no_comunes)

['agron', 'lecrin', 'cordoba', 'santo tome', 'lantejuela  la', 'seron', 'orgiva', 'guevejar', 'chilluevar', 'puebla de cazalla  la', 'villamartin', 'lanjaron', 'huercal overa', 'rambla  la', 'huescar', 'dilar', 'alcaucin', 'montejicar', 'moron de la frontera', 'belmez de la moraleda', 'alcala de guadaira', 'burgo  el', 'almodovar del rio', 'villanueva mesia', 'lupion', 'andujar', 'albanchez de ubeda', 'albunuelas', 'huevar', 'saucejo  el', 'somontin', 'zujar', 'valdepenas de jaen', 'corrales  los', 'alcala del valle', 'villares  los', 'benaojan', 'iruela  la', 'finana', 'benalua de las villas', 'gibraleon', 'iznajar', 'luisiana  la', 'begijar', 'priego de cordoba', 'gastor  el', 'cacin', 'campotejar', 'coin', 'montalban de cordoba', 'puerta de segura  la', 'castro del rio', 'pozo alcon', 'victoria  la', 'montefrio', 'armuna de almanzora', 'escanuela', 'canete de las torres', 'torre alhaquime', 'roda de andalucia  la', 'nivar', 'villaviciosa de cordoba', 'peza  la', 'banos de la encina'

In [60]:
len(elementos_no_comunes)

108